<a href="https://colab.research.google.com/github/rainzy09/Machine_Learning/blob/master/HW1_Intro_PM2_5_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd drive/MyDrive/
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"rainzy","key":"9bd4d254d1010c3369e6b2ecf9d31d60"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

/content/drive/MyDrive


In [2]:
!kaggle competitions download -c ml2020spring-hw1

  0% 0.00/455k [00:00<?, ?B/s]
100% 455k/455k [00:00<00:00, 29.2MB/s]
  0% 0.00/2.25k [00:00<?, ?B/s]
100% 2.25k/2.25k [00:00<00:00, 317kB/s]
  0% 0.00/192k [00:00<?, ?B/s]
100% 192k/192k [00:00<00:00, 26.2MB/s]


In [3]:
!ls


 cogdl		    OpenHINE		    test       train.csv
'Colab Notebooks'   sample_submission.csv   test.csv


In [13]:
import sys
import pandas as pd
import numpy as np
data = pd.read_csv('train.csv', encoding = 'big5')

print(data)

              日期  測站          測項     0     1  ...    19    20    21    22    23
0       2014/1/1  豐原    AMB_TEMP    14    14  ...    16    15    15    15    15
1       2014/1/1  豐原         CH4   1.8   1.8  ...   1.8   1.8   1.8   1.8   1.8
2       2014/1/1  豐原          CO  0.51  0.41  ...  0.45  0.38  0.35  0.36  0.32
3       2014/1/1  豐原        NMHC   0.2  0.15  ...  0.12   0.1  0.09   0.1  0.08
4       2014/1/1  豐原          NO   0.9   0.6  ...   1.9   1.5   1.6   1.8   1.5
...          ...  ..         ...   ...   ...  ...   ...   ...   ...   ...   ...
4315  2014/12/20  豐原         THC   1.8   1.8  ...   1.9   1.9   1.9     2     2
4316  2014/12/20  豐原       WD_HR    46    13  ...   109   108   114   108   109
4317  2014/12/20  豐原  WIND_DIREC    36    55  ...    97   107   118   100   105
4318  2014/12/20  豐原  WIND_SPEED   1.9   2.4  ...     1   1.7   1.5     2     2
4319  2014/12/20  豐原       WS_HR   0.7   0.8  ...   1.4   1.3   1.6   1.8     2

[4320 rows x 27 columns]


# **Preprocessing** 
预处理，将NR补成0


data.iloc() 解释：

loc函数：通过行索引 "Index" 中的具体值来取行数据（如取"Index"为"A"的行）
iloc函数：通过行号来取行数据（如取第二行的数据）

In [14]:
data = data.iloc[:, 3:] # 取列数据
data[data == 'NR'] = 0 # 赋值为零
raw_data = data.to_numpy() # 转为np
print(raw_data)

[['14' '14' '14' ... '15' '15' '15']
 ['1.8' '1.8' '1.8' ... '1.8' '1.8' '1.8']
 ['0.51' '0.41' '0.39' ... '0.35' '0.36' '0.32']
 ...
 ['36' '55' '72' ... '118' '100' '105']
 ['1.9' '2.4' '1.9' ... '1.5' '2' '2']
 ['0.7' '0.8' '1.8' ... '1.6' '1.8' '2']]


# **Extract Features (1)**

训练集 20天 每天24个小时 针对每一天来说，其包含的信息维度为(18,24)(18项指标，24个时间节点)。

將原始 4320 * 18 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料。

In [15]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample
print(month_data)

{0: array([[14.  , 14.  , 14.  , ..., 14.  , 13.  , 13.  ],
       [ 1.8 ,  1.8 ,  1.8 , ...,  1.8 ,  1.8 ,  1.8 ],
       [ 0.51,  0.41,  0.39, ...,  0.34,  0.41,  0.43],
       ...,
       [35.  , 79.  ,  2.4 , ..., 48.  , 63.  , 53.  ],
       [ 1.4 ,  1.8 ,  1.  , ...,  1.1 ,  1.9 ,  1.9 ],
       [ 0.5 ,  0.9 ,  0.6 , ...,  1.2 ,  1.2 ,  1.3 ]]), 1: array([[ 15.  ,  14.  ,  14.  , ...,   8.4 ,   8.  ,   7.6 ],
       [  1.8 ,   1.8 ,   1.7 , ...,   1.7 ,   1.7 ,   1.7 ],
       [  0.27,   0.26,   0.25, ...,   0.36,   0.35,   0.32],
       ...,
       [113.  , 109.  , 104.  , ...,  72.  ,  65.  ,  69.  ],
       [  2.3 ,   2.2 ,   2.6 , ...,   1.9 ,   2.9 ,   1.5 ],
       [  2.5 ,   2.2 ,   2.2 , ...,   0.9 ,   1.6 ,   1.1 ]]), 2: array([[ 18.  ,  18.  ,  18.  , ...,  14.  ,  13.  ,  13.  ],
       [  1.8 ,   1.8 ,   1.8 , ...,   1.8 ,   1.8 ,   1.8 ],
       [  0.39,   0.36,   0.4 , ...,   0.42,   0.47,   0.49],
       ...,
       [103.  , 128.  , 115.  , ...,  60.  ,  94.  ,  53

每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data(数据集)，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。

對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)

In [22]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x.ndim)
#print(y)

2


In [23]:
print(x.shape)

(5652, 162)



# **Normalize (1)**

归一化 

Normalize的方法我们选用最常用的标准差标准化方法这是说明：

数据的标准化（normalization）是将数据按比例缩放，使之落入一个小的特定区间。
在某些比较和评价的指标处理中经常会用到，去除数据的单位限制，将其转化为无量纲的纯数值，
便于不同单位或量级的指标能够进行比较和加权。
最常见的标准化方法就是Z标准化，也是SPSS中最为常用的标准化方法，spss默认的标准化方法就是z-score标准化。
也叫标准差标准化，这种方法给予原始数据的均值（mean）和标准差（standard deviation）进行数据的标准化。
经过处理的数据符合标准正态分布，即均值为0，标准差为1，注意，一般来说z-score不是归一化，而是标准化，归一化只是标准化的一种[lz]。
其转化函数为：
x* = (x - μ ) / σ
其中μ为所有样本数据的均值，σ为所有样本数据的标准差。
具体可看这个https://www.cnblogs.com/chenyusheng0803/p/9867579.html

In [24]:
mean_x = np.mean(x, axis = 0) #18 * 9 求均值，axis = 0表示对各列求均值，返回 1* 列数 的矩阵
std_x = np.std(x, axis = 0) #18 * 9  求标准差，axis = 0表示对各列求均值，返回 1* 列数 的矩阵
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]



接下来是将训练数据按8:2拆成训练数据和验证数据。这样的好处是因为最终只给我们test data的输入而没有给我们输出，所以我们无法定量我们模型的好坏，而使用验证数据可以简单验证我们模型的好坏，让我们自己心里有数。

In [25]:

import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

# **Training**

训练使用的Loss function和梯度下降算法。

loss function我们选择的是Root Mean Square Error，即下图所示的公式：


至于学习率，最简单的办法就是固定学习率，每次迭代的学习率都相同，但这样效果也会很差，良好的学习率应随迭代次数依次减少，所以我们使用自适应学习率的adagrad算法，即每次学习率都等于其除以之前所有的梯度平方和再开根号，即下图所示的公式:


gradient就没什么好说的了，就是一个对Loss函数进行每个参数的偏微分而得到的矩阵。

In [26]:
dim = 18 * 9 + 1 # 用来做参数vector的维数，加1是为了对bias好处理（还有个误差项）。即最后的h(x)=w1x1+w2x2+'''+WnXn+b
w = np.zeros([dim, 1]) # 生成一个dim行1列的数组用来保存参数值，对比源码我这里改成了ones而不是zeros
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float) # np.ones来生成12*471行1列的全1数组，np.concatenate，axis=1表示按列将两个数组拼接起来，即在x最前面新加一列内容，之前x是12*471行18*9列的数组，新加一列之后变为12*471行18*9+1列的数组
learning_rate = 100 # 学习率
iter_time = 1000 # 迭代次数 # 每一百次迭代就输出其损失
adagrad = np.zeros([dim, 1]) # 生成dim行即163行1列的数组，用来使用adagrad算法更新学习率
eps = 0.0000000001 # 因为新的学习率是learning_rate/sqrt(sum_of_pre_grads**2),而adagrad=sum_of_grads**2,所以处在分母上而迭代时adagrad可能为0，所以加上一个极小数，使其不除0
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse # rmse loss函数是从0-n的(X*W-Y)**2之和/(471*12)再开根号，即使用均方根误差(root mean square error),具体可百度其公式，/471/12即/N(次数)
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

0:27.071214829194115
100:33.78905859777454
200:19.91375129819709
300:13.53106819368969
400:10.64546615844617
500:9.277353455475065
600:8.5180420459565
700:8.014061987588422
800:7.63675682477569
900:7.3365637403711235


array([[ 2.13740269e+01],
       [ 3.58888909e+00],
       [ 4.56386323e+00],
       [ 2.16307023e+00],
       [-6.58545223e+00],
       [-3.38885580e+01],
       [ 3.22235518e+01],
       [ 3.49340354e+00],
       [-4.60308671e+00],
       [-1.02374754e+00],
       [-3.96791501e-01],
       [-1.06908800e-01],
       [ 2.22488184e-01],
       [ 8.99634117e-02],
       [ 1.31243105e-01],
       [ 2.15894989e-02],
       [-1.52867263e-01],
       [ 4.54087776e-02],
       [ 5.20999235e-01],
       [ 1.60824213e-01],
       [-3.17709451e-02],
       [ 1.28529025e-02],
       [-1.76839437e-01],
       [ 1.71241371e-01],
       [-1.31190032e-01],
       [-3.51614451e-02],
       [ 1.00826192e-01],
       [ 3.45018257e-01],
       [ 4.00130315e-02],
       [ 2.54331382e-02],
       [-5.04425219e-01],
       [ 3.71483018e-01],
       [ 8.46357671e-01],
       [-8.11920428e-01],
       [-8.00217575e-02],
       [ 1.52737711e-01],
       [ 2.64915130e-01],
       [-5.19860416e-02],
       [-2.5

# **Testing**

In [27]:
w = np.load('weight.npy')
# 使用x_validation和y_validation来计算模型的准确率，因为X已经normalize了，所以不需要再来一遍，只需在x_validation上添加新的一列作为bias的乘数即可
x_validation = np.concatenate((np.ones([1131, 1]), x_validation), axis=1).astype(float)
ans_y = np.dot(x_validation, w)
loss = np.sqrt(np.sum(np.power(ans_y - y_validation, 2)) / 1131)
print(loss)

6.346049158054552


In [29]:
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3093: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

In [30]:
# 进行预测 
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)  # test data的预测值ans_y=test_x与W的积
ans_y

array([[ 5.17496040e+00],
       [ 1.83062143e+01],
       [ 2.04912181e+01],
       [ 1.15239429e+01],
       [ 2.66160568e+01],
       [ 2.05313481e+01],
       [ 2.19065510e+01],
       [ 3.17364687e+01],
       [ 1.33916741e+01],
       [ 6.44564665e+01],
       [ 2.02645688e+01],
       [ 1.53585761e+01],
       [ 6.85894728e+01],
       [ 4.84281137e+01],
       [ 1.87023338e+01],
       [ 1.01885957e+01],
       [ 3.07403629e+01],
       [ 7.11322178e+01],
       [-4.13051739e+00],
       [ 1.82356940e+01],
       [ 3.85789223e+01],
       [ 7.13115197e+01],
       [ 7.41034816e+00],
       [ 1.87179553e+01],
       [ 1.49372503e+01],
       [ 3.67197367e+01],
       [ 1.79616970e+01],
       [ 7.57894629e+01],
       [ 1.23093102e+01],
       [ 5.62953517e+01],
       [ 2.51131609e+01],
       [ 4.61024867e+00],
       [ 2.48377055e+00],
       [ 2.47594223e+01],
       [ 3.04802805e+01],
       [ 3.84639307e+01],
       [ 4.42023106e+01],
       [ 3.00868360e+01],
       [ 4.0

In [31]:
# 将预测结果填入文件当中
import csv

with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 5.174960398984736]
['id_1', 18.306214253527905]
['id_2', 20.491218094180542]
['id_3', 11.523942869805357]
['id_4', 26.61605675230614]
['id_5', 20.531348081761216]
['id_6', 21.906551018797387]
['id_7', 31.736468747068837]
['id_8', 13.391674055111729]
['id_9', 64.45646650291954]
['id_10', 20.26456883615944]
['id_11', 15.358576077361231]
['id_12', 68.58947276926726]
['id_13', 48.42811374745719]
['id_14', 18.702333824193214]
['id_15', 10.188595737466702]
['id_16', 30.74036285982045]
['id_17', 71.13221776355105]
['id_18', -4.130517391262438]
['id_19', 18.23569401642869]
['id_20', 38.578922275007756]
['id_21', 71.31151972531332]
['id_22', 7.410348162634086]
['id_23', 18.717955330321395]
['id_24', 14.937250260084578]
['id_25', 36.719736694705304]
['id_26', 17.961697005662725]
['id_27', 75.78946287210537]
['id_28', 12.30931024861448]
['id_29', 56.29535173964959]
['id_30', 25.113160865661477]
['id_31', 4.610248674094039]
['id_32', 2.4837705545150315]
['id_33', 24.759422